# Create Vinnova Awards

Creates Vinnova (Sweden's Innovation Agency) awards from SWECRIS open data. ~24K projects.

**Prerequisites:**
- Run `scripts/local/vinnova_to_s3.py` to process and upload the data first.

**Data source:** https://www.vr.se/english/swecris.html (filtered by Vinnova)  
**S3 location:** `s3a://openalex-ingest/awards/vinnova/vinnova_projects.parquet`

**Vinnova funder:**
- funder_id: 4320321030
- display_name: "VINNOVA"
- ROR: https://ror.org/01kd5m353
- DOI: 10.13039/501100001858

## Step 1: Create Staging Table from S3

In [ ]:
%sql
-- Create the staging table from S3 parquet
CREATE OR REPLACE TABLE openalex.awards.vinnova_raw
USING delta
AS
SELECT
    *,
    current_timestamp() as databricks_ingested_at
FROM parquet.`s3a://openalex-ingest/awards/vinnova/vinnova_projects.parquet`;

In [ ]:
%sql
-- Check row count (should be ~24K)
SELECT COUNT(*) as total_projects FROM openalex.awards.vinnova_raw;

In [ ]:
%sql
-- Inspect the schema
DESCRIBE openalex.awards.vinnova_raw;

In [ ]:
%sql
-- Sample the raw data
SELECT * FROM openalex.awards.vinnova_raw LIMIT 5;

## Step 2: Create Vinnova Awards Table

In [ ]:
%sql
CREATE OR REPLACE TABLE openalex.awards.vinnova_awards
USING delta
AS
WITH
-- Get Vinnova funder from OpenAlex by explicit funder_id
vinnova_funder AS (
    SELECT
        funder_id,
        display_name,
        ror_id,
        doi
    FROM openalex.common.funder
    WHERE funder_id = 4320321030  -- VINNOVA
),

awards_transformed AS (
    SELECT
        -- Generate unique ID using xxhash64 of funder_id:project_id
        abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(TRIM(g.projectid))))) % 9000000000 as id,

        -- Display name = English project title (fallback to Swedish)
        COALESCE(g.projecttitleen, g.projecttitlesv) as display_name,

        -- Description = English abstract (fallback to Swedish)
        COALESCE(g.projectabstracten, g.projectabstractsv) as description,

        -- Funder info
        f.funder_id,
        TRIM(g.projectid) as funder_award_id,

        -- Amount in SEK
        TRY_CAST(g.fundingssek AS DOUBLE) as amount,
        'SEK' as currency,

        -- Funder struct
        struct(
            CONCAT('https://openalex.org/F', f.funder_id) as id,
            f.display_name,
            f.ror_id,
            f.doi
        ) as funder,

        -- Funding type - Vinnova is innovation/research funding
        CASE
            WHEN LOWER(g.typeofawarddescren) LIKE '%grant%' THEN 'grant'
            ELSE 'grant'
        END as funding_type,

        -- Funder scheme = award type
        g.typeofawarddescren as funder_scheme,

        -- Provenance
        'swecris_vinnova' as provenance,

        -- Dates
        TRY_TO_DATE(g.projectstartdate) as start_date,
        TRY_TO_DATE(g.projectenddate) as end_date,
        YEAR(TRY_TO_DATE(g.projectstartdate)) as start_year,
        YEAR(TRY_TO_DATE(g.projectenddate)) as end_year,

        -- Lead investigator - extracted from pi_name field
        CASE
            WHEN g.pi_name IS NOT NULL AND TRIM(g.pi_name) != '' THEN
                struct(
                    CAST(NULL AS STRING) as given_name,
                    TRIM(g.pi_name) as family_name,  -- Store full name in family_name
                    CASE 
                        WHEN g.pi_orcid IS NOT NULL AND TRIM(g.pi_orcid) != '' 
                        THEN CONCAT('https://orcid.org/', TRIM(g.pi_orcid))
                        ELSE NULL 
                    END as orcid,
                    TRY_TO_DATE(g.projectstartdate) as role_start,
                    struct(
                        TRIM(g.coordinatingorganisationnameen) as name,
                        'Sweden' as country,
                        CAST(NULL AS ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>) as ids
                    ) as affiliation
                )
            ELSE NULL
        END as lead_investigator,

        -- Co-lead and other investigators (not available at project level in SWECRIS)
        CAST(NULL AS STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >) as co_lead_investigator,

        CAST(NULL AS ARRAY<STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >>) as investigators,

        -- Landing page URL - SWECRIS
        'https://www.vr.se/english/swecris.html' as landing_page_url,

        -- No DOI for Vinnova projects
        CAST(NULL AS STRING) as doi,

        -- Works API URL
        concat('https://api.openalex.org/works?filter=awards.id:G', abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(TRIM(g.projectid))))) % 9000000000) as works_api_url,

        -- Timestamps
        current_timestamp() as created_date,
        current_timestamp() as updated_date

    FROM openalex.awards.vinnova_raw g
    CROSS JOIN vinnova_funder f
    WHERE g.projectid IS NOT NULL
      AND TRIM(g.projectid) != ''
)

SELECT * FROM awards_transformed;

In [ ]:
%sql
-- Insert into openalex_awards_raw with priority
INSERT INTO openalex.awards.openalex_awards_raw
SELECT
    id,
    display_name,
    description,
    funder_id,
    funder_award_id,
    amount,
    currency,
    funder,
    funding_type,
    funder_scheme,
    provenance,
    start_date,
    end_date,
    start_year,
    end_year,
    lead_investigator,
    co_lead_investigator,
    investigators,
    landing_page_url,
    doi,
    works_api_url,
    created_date,
    updated_date,
    9 as priority
FROM openalex.awards.vinnova_awards;

## Verification Queries

In [ ]:
%sql
-- Check row count (should be ~24K)
SELECT COUNT(*) as total_vinnova_awards FROM openalex.awards.vinnova_awards;

In [ ]:
%sql
-- Sample the data
SELECT 
    id,
    display_name,
    funder_award_id,
    funder_scheme,
    funding_type,
    amount,
    currency,
    start_date,
    lead_investigator
FROM openalex.awards.vinnova_awards 
LIMIT 10;

In [ ]:
%sql
-- Check funder distribution (should all be VINNOVA)
SELECT funder.display_name, COUNT(*) as cnt
FROM openalex.awards.vinnova_awards
GROUP BY funder.display_name
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funder_scheme distribution
SELECT funder_scheme, COUNT(*) as cnt
FROM openalex.awards.vinnova_awards
WHERE funder_scheme IS NOT NULL
GROUP BY funder_scheme
ORDER BY cnt DESC
LIMIT 10;

In [ ]:
%sql
-- Check data completeness
SELECT
    COUNT(*) as total,
    COUNT(display_name) as has_title,
    COUNT(description) as has_abstract,
    COUNT(amount) as has_amount,
    COUNT(start_date) as has_start_date,
    COUNT(lead_investigator) as has_pi,
    SUM(CASE WHEN lead_investigator.orcid IS NOT NULL THEN 1 ELSE 0 END) as has_orcid,
    ROUND(COUNT(display_name) * 100.0 / COUNT(*), 1) as pct_title,
    ROUND(COUNT(description) * 100.0 / COUNT(*), 1) as pct_abstract,
    ROUND(COUNT(amount) * 100.0 / COUNT(*), 1) as pct_amount,
    ROUND(COUNT(start_date) * 100.0 / COUNT(*), 1) as pct_dates
FROM openalex.awards.vinnova_awards;

In [ ]:
%sql
-- Check year distribution
SELECT start_year, COUNT(*) as cnt
FROM openalex.awards.vinnova_awards
WHERE start_year IS NOT NULL
GROUP BY start_year
ORDER BY start_year DESC
LIMIT 15;

In [ ]:
%sql
-- Check lead institutions (top 15)
SELECT 
    lead_investigator.affiliation.name as institution,
    COUNT(*) as cnt
FROM openalex.awards.vinnova_awards
WHERE lead_investigator.affiliation.name IS NOT NULL
GROUP BY lead_investigator.affiliation.name
ORDER BY cnt DESC
LIMIT 15;

In [ ]:
%sql
-- Check total funding amount
SELECT 
    ROUND(SUM(amount) / 1000000000, 2) as total_funding_billions_sek,
    ROUND(AVG(amount), 0) as avg_award_sek,
    ROUND(PERCENTILE_APPROX(amount, 0.5), 0) as median_award_sek
FROM openalex.awards.vinnova_awards
WHERE amount IS NOT NULL;